In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import random
#import gurobipy as gp
#from gurobipy import GRB


In [4]:
df_exR = pd.read_csv("exR_file.csv")
df_exR
ticker = list(df_exR)[1:] #the first is the time stamp
print(ticker)

['XSP.TO', 'XIU.TO', 'XIN.TO', 'XBB.TO', 'XGB.TO', 'XRB.TO', 'XRE.TO']


In [5]:
#need to transfer to main code
df1= df_exR.iloc[:,1:2]
df2 = df_exR.iloc[:,2:3]
df3 = df_exR.iloc[:,3:6]
df4 = df_exR.iloc[:,6:8]
asset_1_index = [0]
asset_2_index = list(range(1,2))
asset_3_index = list(range(2,5))
asset_4_index = list(range(5,7))
'''
print(df1)
print(df2)
asset_2_index
asset_3_index
asset_4_index
#np.array(ticker)[asset_4_index]
type(asset_2_index)
'''


total_df = pd.concat([df1,df2,df3,df4], axis=1)# merge the 4 asset class
#print(total_df)
#print(total_df.shape)

#print(total_df.shape[1]) # get the amount of columns of the dataframe
numAsset = total_df.shape[1]
print("number of asset is ", numAsset)
Q = total_df.cov()

'\nprint(df1)\nprint(df2)\nasset_2_index\nasset_3_index\nasset_4_index\n#np.array(ticker)[asset_4_index]\ntype(asset_2_index)\n'

In [8]:
def formDf(x,ticker):
    x = pd.DataFrame(x).T
    x.columns = ticker
    return x

In [27]:
# Equal Weight strategy for Block
def EW(numAsset):
    weight_eq = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    x_ew = formDf(weight_eq,ticker) # result for equal weight
    return x_ew

EW(numAsset)

,XSP.TO,XIU.TO,XIN.TO,XBB.TO,XGB.TO,XRB.TO,XRE.TO
0,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857


In [34]:
# Mean Variance optimization Block

def MVO(numAsset):
    
    def meanVarianceOpt(numAsset):
        def objective(weight):#weight
            weight = np.array(weight)
            return weight.dot(Q).dot(weight.T)

        #constraints
        cons = ({'type':"eq",'fun':lambda x: np.sum(x)-1},
               )

        #long only
        bounds = tuple((0,1) for x in range(numAsset))

        #initialize the weights with an even split, equal weights
        guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
        optimized_result = minimize(objective, guess, method = "SLSQP", bounds  = bounds, constraints= cons)

        return optimized_result
    x_mvo = meanVarianceOpt(numAsset).x
    x_mvo = formDf(x_mvo,ticker)
    return x_mvo

weight_mvo = MVO(numAsset) #temperarily set for current data set


In [37]:
# Mean Variance optimization - All weather Block

def MVO_AW(numAsset):
    
    def meanVarianceOpt_allWeather(numAsset):
        def objective(weight):#weight
            weight = np.array(weight)
            return weight.dot(Q).dot(weight.T)

        #constraints
        cons = ({'type':"eq",'fun':lambda x: np.sum(x)-1},
                {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_1_index])-0.15 },
                {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_2_index])-0.15 },
                {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_3_index])-0.15 },
                {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_4_index])-0.15 },
                {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_1_index]) },
                {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_2_index]) },
                {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_3_index]) },
                {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_4_index]) },
               )

        #long only
        bounds = tuple((0,0.3) for x in range(numAsset))

        #initialize the weights with an even split, equal weights
        guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
        optimized_result = minimize(objective, guess, method = "SLSQP", bounds  = bounds, constraints= cons)

        return optimized_result
    x_mvo_allWeather = meanVarianceOpt_allWeather(numAsset).x
    x_mvo_allWeather = formDf(x_mvo_allWeather, ticker)
    return x_mvo_allWeather

weight_mvo_aw = MVO_AW(numAsset)

In [39]:
# Naive Risk Parity Block
def naive_rp(Q,numAsset):
    
    sigma_sq = np.diagonal(Q)# variance for all assets (sigma^2 )
    #1/sigma^2
    inv_sigma_sq = pd.DataFrame(sigma_sq).rdiv(1).T 

    # 1/sigma
    inv_sigma = inv_sigma_sq.apply(np.sqrt)
    inv_sigma= np.array(inv_sigma.iloc[0,:].tolist())
    
    x_naive_rp = []
    denom = np.sum(inv_sigma) #sum of 1/sigma at the denominator

    for i in range(numAsset):
        x_naive_rp.append((inv_sigma[i]/denom))

    x_naive_rp = formDf(x_naive_rp, ticker)
    return x_naive_rp

weight_naive_rp = naive_rp(Q , numAsset)

In [40]:
weight_naive_rp

,XSP.TO,XIU.TO,XIN.TO,XBB.TO,XGB.TO,XRB.TO,XRE.TO
0,0.075728,0.087991,0.078344,0.272712,0.275387,0.135146,0.074693


In [44]:
# Risk Parity


#define initial portfolio (equal weighed)
#kappa = 5 #initial arbitrary constant
#y0 = np.transpose(np.mat((np.repeat(1/numAsset,numAsset))))


22.168500174797572

In [ ]:
#create a new model
#model_1 = gp.Model("matrix1")

#create variables
#y = model_1.addMVar(shape = 7, vtype=GRB.CONTINUOUS,lb = 0,name ="y" )

#set objective
#obj = 